In [1]:
from preproc import preprocess_rsc15
dat, ind2val = preprocess_rsc15()

START load data
Loaded train set
	Events: 31637239
	Sessions: 7966257
	Items: 37483
	Span: 2014-04-01 / 2014-09-29

Loaded test set
	Events: 71222
	Sessions: 15324
	Items: 6751
	Span: 2014-09-29 / 2014-09-30

END load data  15.717937 c /  15.721532344818115 s


## Spotlight models

In [5]:
dat['train'].head()

,SessionId,ItemId,Time,SessionId_idx,ItemId_idx
0,1,214536502,1.396868e+09,1,1720
1,1,214536500,1.396868e+09,1,1719
2,1,214536506,1.396868e+09,1,1721
3,1,214577561,1.396868e+09,1,7872
4,2,214662742,1.396879e+09,2,14845


In [13]:
# Transform into sequence interaction object
from spotlight.interactions import Interactions, SequenceInteractions
dat_seq = {}
for name, df in dat.items():
    dat_seq[name] = Interactions(user_ids=df.SessionId_idx.values,
                item_ids=df.ItemId_idx.values,
                timestamps=df.Time.values).to_sequence()

In [ ]:
np.random.seed(2)
idx = np.random.choice(dat_seq['test'].sequences.shape[0], size=5000, replace=False)
smalltest = SequenceInteractions(sequences=dat_seq['test'].sequences[idx])

In [72]:
from spotlight.sequence.implicit import ImplicitSequenceModel
from spotlight.evaluation import sequence_mrr_score, sequence_precision_recall_score

losses = ['pointwise', 'bpr', 'hinge', 'adaptive_hinge']
reps = ['pooling', 'cnn', 'lstm', 'mixture']

for l in losses:
    for net in reps:
        model = ImplicitSequenceModel(n_iter=3,
                                      representation=net,
                                      loss='bpr',
                                      use_cuda=True)
        model.fit(dat_seq['train'], verbose = True)

        mrr_test = sequence_mrr_score(model,smalltest)
        print(f"loss: {l}, net: {net}, mrr: {mrr_test.mean()}")

Epoch 0: loss 0.03826883341834484
Epoch 1: loss 0.027478126867439997
Epoch 2: loss 0.026426138140883543
loss: pointwise, net: pooling, mrr: 0.23221806320220012
Epoch 0: loss 0.04500080469874677
Epoch 1: loss 0.033241427644791544
Epoch 0: loss 0.08078527066152529
Epoch 1: loss 0.06289324530504865
Epoch 2: loss 0.04281303559503388
loss: pointwise, net: lstm, mrr: 0.028338055202922312
Epoch 0: loss 0.08575799986690696
Epoch 1: loss 0.07207300414489477
Epoch 2: loss 0.0675103946596535
loss: pointwise, net: mixture, mrr: 0.004433035005343094
Epoch 0: loss 0.038293197370767654
Epoch 1: loss 0.027557578657817036
Epoch 2: loss 0.026457562711702836
loss: bpr, net: pooling, mrr: 0.22839831490281776
Epoch 0: loss 0.04473836292980764
Epoch 1: loss 0.03316664076560146
Epoch 2: loss 0.031379143218974004
loss: bpr, net: cnn, mrr: 0.19435913019648582
Epoch 0: loss 0.07229191405199552
Epoch 1: loss 0.04372591415695019
Epoch 2: loss 0.03732955171989026
loss: bpr, net: lstm, mrr: 0.045946355262809775
Epo

In [22]:
mrr_test.mean()

0.18466458649339706

In [ ]:
11

In [ ]:
1